In [ ]:
# Import packages
import pyspark
import dxpy
import dxdata

In [ ]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [ ]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [ ]:
participant = dataset['participant']

In [ ]:
field_name_dict = {
    'sample_names': 'eid',
    'age_assessment0': 'p21003_i0',
    'age_assessment1': 'p21003_i1',
    'age_assessment2': 'p21003_i2',
    'age_assessment3': 'p21003_i3',
    'ethnic_background0': 'p21000_i0',
    'ethnic_background1': 'p21000_i1',
    'ethnic_background2': 'p21000_i2',
    'genetic_kinship_to_other_participants': 'p22021',
    'sex': 'p31',
    'genetic_sex': 'p22001',
    'bmi0': 'p21001_i0',
    'bmi1': 'p21001_i1',
    'bmi2': 'p21001_i2',
    'bmi3': 'p21001_i3',
    'bmi_prs': 'p26216',
    'hba1c_df0': 'p30750_i0',
    'hba1c_df1': 'p30750_i1',
    'hba1c_df_prs': 'p26238',
    'hdl0': 'p30760_i0',
    'hdl1': 'p30760_i1',
    'hdl_prs': 'p26242',
    'ldl_sf0': 'p30780_i0',
    'ldl_sf1': 'p30780_i1',
    'ldl_sdf_prs': 'p26250',
    'tch0': 'p23400_i0',
    'tch1': 'p23400_i1',
    'tch_prs': 'p26280', # Enhanced PRS for TCH
    'ttg0': 'p23407_i0',
    'ttg1': 'p23407_i1',
    'ttg_prs': 'p26282', # Enhanced PRS for TTG 
    'cvd_prs': 'p26223',
    'cad_prs': 'p26227',
    'ht_prs': 'p26244',
    't1d_prs': 'p26283',
    't2d_prs': 'p26285',
}


for idx in range(1, 11):
    field_name_dict[f'genetic_pca{idx}'] = f'p22009_a{idx}'

In [ ]:
field_names = list(field_name_dict.values())

In [ ]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")

In [ ]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return

In [ ]:
pandas_df = df.toPandas()

In [ ]:
pandas_df = pandas_df.rename(columns={v:k for k,v in field_name_dict.items()})

In [ ]:
for col in field_name_dict.keys():
    try:
        pandas_df[col] = pandas_df[col].astype(float)
    except ValueError:
        continue

In [ ]:
start = 0
final = len(pandas_df)
block = 0
proj_dir = f"/fields/data/bmi_and_assoc_quant_traits/"
nrows = 50000

while start<final:
    end = start + nrows
    table = pandas_df.iloc[start:end, :]
    filename = f"block{block}.csv.gz"
    table.to_csv(filename, index=False)
    upload_file_to_project(filename, proj_dir)
    start += nrows
    block += 1